In [ ]:
help(exec)

In [ ]:
globals

In [ ]:
import re

def read_settings(fn):
    content = open(fn).read()
    globals = {}
    locals = {}
    exec(content, globals, locals)
    return { k:v for k,v in locals.items() if re.match("[A-Z]+.*", k) }

In [ ]:
dev = read_settings("config_dev.py")

In [ ]:
shared = read_settings("config_global.py")

In [ ]:
prod = read_settings("config_prod.py")

In [ ]:
set(dev.keys()).difference(prod.keys())

In [ ]:
set(prod.keys()).difference(dev.keys())

In [ ]:
set(dev.keys()).intersection(shared.keys())

In [ ]:
set(prod.keys()).intersection(shared.keys())

In [ ]:
for k in set(prod.keys()).intersection(dev.keys()):
    if prod[k] == dev[k]:
        shared[k] = prod[k]
        del prod[k]
        del dev[k]

In [ ]:
prod.keys()

In [ ]:
shared.keys()

In [ ]:
config_vars = set(shared.keys()) # set(prod.keys()).union(shared.keys()).union(dev.keys())

In [ ]:
def rewrite(fn):
    constant_var_pattern = re.compile("("+("|".join(config_vars))+")")
    env_var_pattern = re.compile("("+("|".join(set(prod.keys()).union(dev.keys())))+")")
    
    print("rewriting", fn)
    content = open(fn, "rt").read()
    assert "constants" not in content
    with open(f"{fn}", "wt") as fd:
        new_content = re.sub(constant_var_pattern, "constants.\\1", content)
        new_content = re.sub(env_var_pattern, "env_config.\\1", new_content)
        new_content = re.sub("from depmapomics.config import \\*", "", new_content)
        if new_content != content:
            fd.write("from depmapomics import constants\n")
            fd.write("from depmapomics import env_config\n")
        fd.write(new_content)

with open("constants.py", "wt") as fd:
    for k, v in shared.items():
        fd.write(f"{k} = {repr(v)}\n\n")

with open("config_dev.py", "wt") as fd:
    for k, v in dev.items():
        fd.write(f"{k} = {repr(v)}\n\n")

with open("config_prod.py", "wt") as fd:
    for k, v in prod.items():
        fd.write(f"{k} = {repr(v)}\n\n")



In [ ]:
import os

In [ ]:
python_files = []
for root, subdirs, files in os.walk('.'):
    for fn in files:
        if fn.endswith(".py"):
            python_files.append(os.path.join(root, fn))
python_files

In [ ]:
for fn in python_files:
    if fn not in ['./config.py', "./config_dev.py", "./config_global.py", "./config_prod.py", "./constants.py"]:
        rewrite(fn)

In [ ]:
help(re.sub)